In [ ]:
# Interesting error finding in X-ray images. Seem by visual inspection work pretty well. //Per Lindholm
!pip install chainer

In [ ]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer.backends.cuda import to_cpu,to_gpu
import numpy as np
import pylab as plt
import time
import h5py
from sklearn.preprocessing import LabelBinarizer
from skimage.draw import disk
from skimage.transform import resize
import pandas as pd
import glob
from skimage.io import imread
import pydicom
from pydicom.filereader import dcmread

In [ ]:
# !pip install chainer

In [ ]:
df = pd.DataFrame(data=pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')[['id','boxes']]).dropna()


In [ ]:
files = !find ../input/siim-covid19-detection/train/ -name *.dcm

In [ ]:
from time import sleep
dataset = []
i = 0
while i<100:
    sleep(0.01)
    try:
        dataset = dataset + [F.resize_images(np.expand_dims(np.expand_dims(dcmread(files[i]).pixel_array,axis=0),axis=0).astype(np.float32)/32768,(256,256)).array.reshape(256,256)]
        print(i)
    except:
        i = i+1
    i +=1
    

In [ ]:
len(dataset)

In [ ]:
plt.imshow(dataset[30],cmap=plt.cm.gray)

In [ ]:
dataset[0].max()

In [ ]:
N = 20
mask = []
radius = 75
rr = 300
t = np.linspace(0,2*np.pi,N)
for i in range(N):
    r = rr * np.cos(t[i]) + 28*28/2
    c = rr * np.sin(t[i]) + 28*28/2
    z = np.random.rand(784,784)
    z[disk((r,c),radius)] = 1.0
    mask.append(resize(z,(28,28),anti_aliasing=True))


In [ ]:
plt.imshow(np.sum(mask,axis=0))
plt.show()
plt.imshow(mask[1])
plt.show()
plt.imshow(mask[-1])
plt.show()

In [ ]:
class Model2(chainer.Chain):
    def __init__(self,hidden,out):
        super(Model2, self).__init__()
        with self.init_scope():
            self.l0 = L.Linear(None,28*28)
            self.l1 = L.Linear(None,hidden)
            self.l10 = L.Linear(None,out)
            self.mask = np.copy(mask)
            
    def __call__(self, x):
        x = F.moveaxis(F.stack([F.relu(self.l0(x))*self.mask[i].reshape(-1,28*28) for i in range(N)]),0,1)
        x = F.relu(self.l1(x))
        x = self.l10(x)
        return x


In [ ]:
model = Model2(300,1)
optimizer = chainer.optimizers.SMORMS3()
optimizer.setup(model)

In [ ]:
lb = LabelBinarizer()
lb.fit(np.arange(5))

In [ ]:
sol = []
soly = []

In [ ]:
#predict easy predictabel vs hard to predict

def train(nn):
    maxloss = 0
    i = 0
    while True:
        if i>nn:break
        
        idx = np.random.randint(0,92)
        ix = np.random.randint(64,256)
        iy = np.random.randint(64,256)
        
        X = dataset[idx]
        x = X[ix-64:ix,iy-64:iy].reshape(-1,1)
        y = model(x)
        
        loss = F.mean_squared_error(y,x)
        
        if loss.array > 0.75*maxloss:
            maxloss = loss.array
            sol.append([idx,ix,iy])
            soly.append([idx,y,x])
        
            
        
        model.cleargrads()
        loss.backward()
        optimizer.update()
        
        if i%10==0:
            print(i,loss.item())
        i +=1

In [ ]:
train(100)

In [ ]:
sol

In [ ]:
N = 2
n,ix,iy = sol[N]
print(sol[N])
plt.imshow(dataset[n],cmap=plt.cm.gray)
plt.show()
plt.imshow(dataset[n][ix-64:ix,iy-64:iy],cmap=plt.cm.gray)
plt.show()
plt.imshow(soly[N][1].reshape(64,64).array,cmap=plt.cm.gray)
plt.show()
